# Problem 9.6

## Part a

In [1]:
qui import delim ./data/chapter09/Rind_Bordia_JASP_1996, clear

In [2]:
gen Z =.
qui replace Z = 1 if happyface==1
qui replace Z = 0 if happyface==0

rename tip Y

capture program drop var_difference
program define var_difference, rclass
	sum Y if Z==1, detail
	local var_treat = r(Var)
	sum Y if Z==0, detail
	local var_control = r(Var)
	return scalar vardiff= `var_treat'-`var_control'
end

tsrtest Z r(vardiff): var_difference


(89 missing values generated)






Two-sample randomization test for theta=r(vardiff) of var_difference by Z

Combinations:   5.19137106438e+25 = (89 choose 44)
Assuming null=0
Observed theta: 53.31

Minimum time needed for exact test (h:m:s):  1.66e+18:00:00
Reverting to Monte Carlo simulation.
Mode: simulation (10000 repetitions)

progress: |........................................|

 p=0.23448 [one-tailed test of Ho:  theta(Z==0)<=theta(Z==1)]
 p=0.76542 [one-tailed test of Ho:  theta(Z==0)>=theta(Z==1)]
 p=0.47205 [two-tailed test of Ho:  theta(Z==0)==theta(Z==1)]


In [3]:
//  p-value for var(Y1)>Var(Y0)
di %8.4f r(uppertail) 

  0.2345


In [4]:
//  p-value for var(Y1)<>Var(Y0)
di %8.3f r(twotail) 

   0.472


## Part c

In [5]:
rename female female_factor
gen female = .
replace female = 1 if female_factor==1
replace female = 0 if female_factor==0

gen zfemale = Z*female



(89 missing values generated)

(45 real changes made)

(44 real changes made)



In [6]:
//lmmodelint: regression with interaction between happyface and waitstaff sex
regress Y Z female zfemale


      Source |       SS           df       MS      Number of obs   =        89
-------------+----------------------------------   F(3, 85)        =      9.32
       Model |  3072.39611         3  1024.13204   Prob > F        =    0.0000
    Residual |  9335.52582        85  109.829716   R-squared       =    0.2476
-------------+----------------------------------   Adj R-squared   =    0.2211
       Total |  12407.9219        88  140.999113   Root MSE        =     10.48

------------------------------------------------------------------------------
           Y |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
           Z |  -3.629627   3.163098    -1.15   0.254    -9.918714     2.65946
      female |   6.378199   3.163098     2.02   0.047      .089112    12.66729
     zfemale |   8.887078   4.446646     2.00   0.049     .0459551     17.7282
       _cons |   21.40571   2.286916     9.36   0.

In [7]:
//lmmodel: regression model without interaction
regress Y Z female


      Source |       SS           df       MS      Number of obs   =        89
-------------+----------------------------------   F(2, 86)        =     11.59
       Model |  2633.69091         2  1316.84545   Prob > F        =    0.0000
    Residual |  9774.23103        86  113.653849   R-squared       =    0.2123
-------------+----------------------------------   Adj R-squared   =    0.1939
       Total |  12407.9219        88  140.999113   Root MSE        =    10.661

------------------------------------------------------------------------------
           Y |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
           Z |   .8673363   2.261535     0.38   0.702    -3.628446    5.363119
      female |   10.87516   2.261535     4.81   0.000      6.37938    15.37094
       _cons |   19.05503   1.995134     9.55   0.000     15.08883    23.02122
--------------------------------------------------

In [8]:
scalar coeff_z = _b[Z]
cap drop Y0 Y1
gen Y0 = Y - coeff_z * Z
gen Y1 = Y + coeff_z*(1- Z)


qui regress Y Z female zfemale
qui test zfemale		
global f_obs = r(F)

In [9]:
capture program drop wald_f
program define wald_f, rclass
	tempvar Y_sim zsimfemale
	gen `Y_sim' = Y1 * Z + Y0 * (1 - Z)
	gen  `zsimfemale' = female*Z
	qui reg `Y_sim' Z female `zsimfemale'
	test `zsimfemale'
	return scalar f_sims = r(F)
end

In [10]:
tsrtest Z r(f_sims) using 9_6_fsims.dta, overwrite: wald_f	

Two-sample randomization test for theta=r(f_sims) of wald_f by Z

Combinations:   5.19137106438e+25 = (89 choose 44)
Assuming null=0
Observed theta: 3.994

Minimum time needed for exact test (h:m:s):  6.84e+19:00:00
Reverting to Monte Carlo simulation.
Mode: simulation (10000 repetitions)

progress: |........................................|

 p=0.04740 [one-tailed test of Ho:  theta(Z==0)<=theta(Z==1)]
 p=0.95250 [one-tailed test of Ho:  theta(Z==0)>=theta(Z==1)]
 p=0.04740 [two-tailed test of Ho:  theta(Z==0)==theta(Z==1)]

Saving log file to 9_6_fsims.dta...done.


In [11]:
di %8.4f r(uppertail)

  0.0474
